In [1]:
import polars as pl
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
# import validation_data_input
import plotly.express as px
import toml
from pathlib import Path
import util
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

In [2]:
# %store -r validation_data

# config = validation_data.config.copy()
# hh = validation_data.hh.copy()
# person = validation_data.person.copy()
# df_parcel = validation_data.land_use.copy()
# parcel_geog = validation_data.parcel_geog.copy()

config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'validation_configuration.toml'))
input_config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'input_configuration.toml'))

data = util.ValidationData(config,input_config,['hh', 'person', 'land_use'])

hh = data.hh.to_pandas()
person = data.person.to_pandas()
df_parcel = data.land_use.to_pandas()
# parcel_geog = data.parcel_geog.to_pandas()

conn = create_engine('sqlite:///'+config['model_dir']+'/inputs/db/'+input_config['db_name'])
parcel_geog = pd.read_sql(text("SELECT * FROM "
                    + "parcel_"
                    + input_config["base_year"]
                    + "_geography"), con=conn.connect())

In [3]:
hh = hh.merge(df_parcel, left_on='hhparcel', right_on='parcelid', how='left')

In [4]:
# Group income, hh density, and employment density into 4 groups
var_group = hh.loc[hh['source'] == 'model',['hhincome','emptot_1','hh_1']].quantile([.125, .25, .50, .75])

# var_group

In [5]:
hh['hhincome_group'] = pd.cut(hh['hhincome'],bins=[-9999999.0] + var_group['hhincome'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])
# add hh density groups
hh['hh_density_group'] = pd.cut(hh['hh_1'],bins=[-9999999.0] + var_group['hh_1'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])
# add employment density groups
hh['emp_density_group'] = pd.cut(hh['emptot_1'],bins=[-9999999.0] + var_group['emptot_1'].tolist() + [9999999.0], labels=['very low', 'low', 'medium', 'medium-high', 'high'])


In [6]:
person = person.merge(hh, on=['hhno','source'], how='left')

In [7]:
person = person.merge(parcel_geog, left_on='pspcl', right_on='ParcelID', how='left')
person = person.merge(parcel_geog, left_on='hhparcel', right_on='ParcelID', how='left', suffixes=['_school', '_home'])

In [8]:
df_students = person.loc[person['pstyp']>0, :].copy()

In [9]:
df_students[df_students['source']=='survey'].to_clipboard()

In [10]:
df = df_students.loc[df_students['psaudist'] > 0, :].copy()
df['wt_dist'] = df['psaudist']*df['psexpfac']
df = df.groupby(['source'])[['wt_dist','psexpfac']].sum().reset_index()
df['Mean Distance to School by Car (psaudist)'] = df['wt_dist']/df['psexpfac']
df[['source','Mean Distance to School by Car (psaudist)']]

,source,Mean Distance to School by Car (psaudist)
0,model,6.049612
1,survey,3.963511


In [11]:
# auto ownership in Income groups
def plot_school_location(df:pd.DataFrame, var:str, title_cat:str, sub_name:str):
    df = df.loc[df['psaudist'] > 0, :].copy()
    df['wt_dist'] = df['psaudist']*df['psexpfac']
    df_plot = df.groupby(['source',var])[['wt_dist','psexpfac']].sum().reset_index()
    df_plot['average_wt_psaudist'] = df_plot['wt_dist']/df_plot['psexpfac']

    df_plot_ct = df.groupby(['source',var])[['wt_dist','psexpfac']].count().reset_index(). \
    rename(columns={'psexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',var])

    fig = px.bar(df_plot, x=var, y=['average_wt_psaudist', 'source'], color="source",
                    barmode="group",
                     hover_data=['sample count'],
                    title="School Distance by "+ title_cat)
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text=sub_name)
    fig.update_layout(height=400, width=800, font=dict(size=11),
                      yaxis=dict(tickformat=".2f"))
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".2f"))
    fig.show()

## School Location by School Geography

In [12]:
plot_school_location(df_students, 'CountyName_school', 'School County', 'County')

In [13]:
plot_school_location(df_students, 'rg_proposed_school', 'Regional Geography', 'Regional Geography')

## School Location by Home Geography

In [14]:
plot_school_location(df_students, 'CountyName_home', 'Home County', 'County')

In [15]:
plot_school_location(df_students, 'rg_proposed_home', 'Home Regional Geography', 'Regional Geography')

## School Location by Person/Household Characteristics

In [16]:
plot_school_location(df_students[df_students['pptyp'].isin([5,6,7])], 
                     'pptyp', 'Person Type', 'student type')

In [17]:
plot_school_location(df_students, 'hh_density_group', 'Household Density', 'Household Density Group')

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3461027129.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3461027129.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [18]:
plot_school_location(df_students, 'emp_density_group', 'Employment Density', 'Employment Density Group')

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3461027129.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3461027129.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Distance Bins

In [19]:
# distance to school bins from workplace_location.csv
df_students['distance_to_school_bin'] = pd.cut(df_students['psaudist'], bins=[0,1,2,5,15,9999],
                                    labels=['0-1', '1-2', '2-5',
                                            '5-15', '15+'])
# Create bins: bins of 2 miles up to 60 miles
max_bin = 60
bin_size = 2
df_students['d_school_bin_60mi'] = pd.cut(df_students['psaudist'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

In [20]:
df_students['student_type'] = df_students['pptyp'].map({5: 'University',
                                                        6: 'High School 16+',
                                                        7: 'Child Age 5-15'})

In [21]:
df_plot = df_students.loc[(~df_students['student_type'].isnull()) & (df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','distance_to_school_bin',])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

df_plot_ct = df_students.loc[(~df_students['student_type'].isnull()) & (df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','distance_to_school_bin'])['psexpfac'].count().reset_index().rename(columns={'psexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','distance_to_school_bin'])


fig = px.bar(df_plot, x="distance_to_school_bin", y="percentage", color="source",barmode="group",
             hover_data=['sample count'],
             title="Distance to school")
fig.update_layout(height=400, width=700, font=dict(size=11))
fig.show()

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3754020342.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3754020342.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [22]:
df_plot = df_students.loc[(~df_students['student_type'].isnull()) & (df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','distance_to_school_bin','student_type'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','student_type'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

df_plot_ct = df_students.loc[(~df_students['student_type'].isnull()) & (df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','distance_to_school_bin','student_type'])['psexpfac'].count().reset_index().rename(columns={'psexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','student_type','distance_to_school_bin'])


fig = px.bar(df_plot, x="distance_to_school_bin", y="percentage", color="source",barmode="group",
             facet_col="student_type", #facet_col_wrap=2,
             hover_data=['sample count'],
             title="Distance to school by student type")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=900, font=dict(size=11))
fig.show()



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3061907616.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\3061907616.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [23]:

df_plot = df_students.loc[(~df_students['student_type'].isnull()) & (df_students['d_school_bin_60mi'] != float('nan'))].groupby(['source','student_type','d_school_bin_60mi'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','student_type'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

fig2 = px.line(df_plot, x='d_school_bin_60mi', y="percentage", color="source",
               facet_col="student_type", facet_col_wrap=2,
               title="Distance to school by income group")
fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig2.update_layout(height=400, width=700, font=dict(size=11))
fig2.show()

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\165769165.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [24]:
df_plot = df_students.loc[(df_students['pptyp']) & (df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','hhincome_group','distance_to_school_bin'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','hhincome_group'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

df_plot_ct = df_students.loc[(df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','distance_to_school_bin','hhincome_group'])['psexpfac'].count().reset_index().rename(columns={'psexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','hhincome_group','distance_to_school_bin'])


fig = px.bar(df_plot, x="distance_to_school_bin", y="percentage", color="source",barmode="group",
             facet_col="hhincome_group", facet_col_wrap=2,
             hover_data=['sample count'],
             title="Distance to school by household income")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=900, width=700, font=dict(size=11))
fig.show()

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\2670549958.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\2670549958.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\2670549958.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [25]:

df_plot = df_students.loc[(df_students['pptyp']) & (df_students['d_school_bin_60mi'] != float('nan'))].groupby(['source','hhincome_group','d_school_bin_60mi'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','hhincome_group'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

fig2 = px.line(df_plot, x='d_school_bin_60mi', y="percentage", color="source",
               facet_col="hhincome_group", facet_col_wrap=2,
               title="Distance to school by income group")
fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig2.update_layout(height=400, width=700, font=dict(size=11))
fig2.show()

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\1169327965.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\1169327965.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [26]:
df_plot = df_students.loc[(df_students['pptyp']) & (df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','hh_density_group','distance_to_school_bin'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','hh_density_group'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

df_plot_ct = df_students.loc[(df_students['distance_to_school_bin'] != float('nan'))].groupby(['source','distance_to_school_bin','hh_density_group'])['psexpfac'].count().reset_index().rename(columns={'psexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','hh_density_group','distance_to_school_bin'])


fig = px.bar(df_plot, x="distance_to_school_bin", y="percentage", color="source",barmode="group",
             facet_col="hh_density_group", facet_col_wrap=2,
             hover_data=['sample count'],
             title="Distance to school by household density")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=900, width=700, font=dict(size=11))
fig.show()

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\1231057579.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\1231057579.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\1231057579.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [27]:

df_plot = df_students.loc[(df_students['pptyp']) & (df_students['d_school_bin_60mi'] != float('nan'))].groupby(['source','hh_density_group','d_school_bin_60mi'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','hh_density_group'], group_keys=False)['psexpfac']. \
    apply(lambda x: 100 * x / float(x.sum()))
# df_plot

fig2 = px.line(df_plot, x='d_school_bin_60mi', y="percentage", color="source",
               facet_col="hh_density_group", facet_col_wrap=2,
               title="Distance to school by household density")
fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig2.update_layout(height=400, width=700, font=dict(size=11))
fig2.show()

C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\208177071.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\modeller\AppData\Local\Temp\ipykernel_33720\208177071.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

